# Threshold

Import dependencies

In [1]:
import glob
import cv2
import numpy as np

Load images

In [2]:
images = glob.glob("./*/*/*/[0-9]*[0-9].png")

Apply threshold to each image an save the mask

In [4]:
save_name = "otsu-corrected.png"

for image in images:
    #Construct save path
    save_path = image.replace(image.split("\\")[-1], "")

    #Load image to memory in grayscale
    cached_image = cv2.imread(image, cv2.IMREAD_GRAYSCALE)

    #Detect sensor and set threshold value accordingly
    threshold_value = 54.75 if "FX10" in image else 93.47

    #Apply thresholding to the image
    result = cached_image > threshold_value

    #Save the binary image
    cv2.imwrite('{}{}'.format(save_path, save_name), result.astype(np.uint8)*255)

Evaluate the mask quality and save the results

In [13]:
for image in images:
    image_path = image.replace(image.split("\\")[-1], "")
    control = cv2.imread("{}labelbox_mask_combination.png".format(image_path)).reshape(-1)
    to_be_evaluated = cv2.imread("{}{}".format(image_path, save_name)).reshape(-1)

    stats = {
    "total": control.shape[0],
    "correct" : 0,
    "correct_bg" : 0,
    "correct_fg" : 0,
    "false" : 0,
    "false_bg" : 0,
    "false_fg" : 0,
    }

    for c, e in zip(control, to_be_evaluated):
        if c == e:
            stats["correct"] += 1
            if c == 0:
                stats["correct_bg"] += 1
            else:
                stats["correct_fg"] += 1 
        if c != e:
            stats["false"] += 1
            if c == 0:
                stats["false_bg"] += 1
            else:
                stats["false_fg"] += 1 

    file = open("{}results_evaluation_threshold.txt".format(image_path), "w")
    file.write("{}".format(image.split("\\")[-1]).split(".png")[0] + "\n\n")
   

    for stat in stats:
        file.write("{}: {}\n".format(stat, stats[stat]))
        
    file.write("\n")
    for stat in stats:
        file.write("{} (%): {}\n".format(stat, stats[stat] * 100 / control.shape[0]))

    file.close()

    print("Evaluated: {}".format(image_path))

Evaluated: .\Analysis_BP\FX10\20220121_FX10__ras1_1-15_A_2022-01-21_09-12-35\
Evaluated: .\Analysis_BP\FX10\20220121_FX10__ras1_1-15_B_2022-01-21_09-15-30\
Evaluated: .\Analysis_BP\FX10\20220121_FX10__ras1_16-30_A_2022-01-21_09-20-15\
Evaluated: .\Analysis_BP\FX10\20220121_FX10__ras1_16-30_B_2022-01-21_09-22-51\
Evaluated: .\Analysis_BP\FX10\20220121_FX10__ras1_31-40_A_2022-01-21_10-31-47\
Evaluated: .\Analysis_BP\FX10\20220121_FX10__ras1_31-40_B_2022-01-21_10-33-32\
Evaluated: .\Analysis_BP\FX10\20220121_FX10__ras2_1-15_A_2022-01-21_09-25-15\
Evaluated: .\Analysis_BP\FX10\20220121_FX10__ras2_1-15_B_2022-01-21_09-28-37\
Evaluated: .\Analysis_BP\FX10\20220121_FX10__ras2_16-30_A_2022-01-21_09-30-52\
Evaluated: .\Analysis_BP\FX10\20220121_FX10__ras2_16-30_B_2022-01-21_09-34-00\
Evaluated: .\Analysis_BP\FX10\20220121_FX10__ras2_31-40_A_2022-01-21_10-35-17\
Evaluated: .\Analysis_BP\FX10\20220121_FX10__ras2_31-40_B_2022-01-21_10-37-09\
Evaluated: .\Analysis_BP\FX10\20220121_FX10__ras3_1-15_A